### Matrix Diagonalization
Consider a $n \times n$ matrix $A_{n, n}$ with $n$ linearly independent eigenvectors.
<br> Let $S_{n,n}$ be a matrix with these eigenvectors as its columns.
<br>Then $A_{n,n}$ can be factorized as
$$
A = S \Sigma S^{-1}
$$
where $\Sigma$ is a diagonal matrix (all off-digonal elements are zero).
<br> The above matrix factorization is known as diagonalization.

In [17]:
import numpy as np

# Let us reconsider our rotation matrix
def diagonalise(matrix):
    try:
        # np.linalg.eig(A) returns the eigenvalues of A in an
        # array (first return value) and the eigvectors as a
        # matrix (each column is an eigenvector)
        l, e = np.linalg.eig(A)
        
        # make a diagonal matrix from the eigenvalues
        sigma = np.diag(l)
        
        # return the three factor matrices
        return e, np.diag(l), np.linalg.inv(e)
    except np.linalg.LinAlgError:
        print("Cannot diagonalise matrix!")

    
A = np.array([[0.7071, 0.7071, 0], [-0.7071, 0.7071, 0],
              [0, 0, 1]])
print("A =\n{}".format(A))
S, sigma, S_inv = diagonalise(A)

# Let us reconstruct the original matriox from its factors
A1 = np.matmul(S, np.matmul(sigma, S_inv))
print("\nS =\n{}".format(S))
print("\nsigma =\n{}".format(sigma))
print("\nS_inv =\n{}".format(S_inv))
print("\nS sigmaa S_inv =\n{}".format(A1))

# We assert that the original matrix is the same as the
# reconstruction from the diagonal decomposition factors.
assert np.allclose(A, A1)

A =
[[ 0.7071  0.7071  0.    ]
 [-0.7071  0.7071  0.    ]
 [ 0.      0.      1.    ]]

S =
[[ 0.70710678+0.j          0.70710678-0.j          0.00000000+0.j        ]
 [ 0.00000000+0.70710678j  0.00000000-0.70710678j  0.00000000+0.j        ]
 [ 0.00000000+0.j          0.00000000-0.j          1.00000000+0.j        ]]

sigma =
[[ 0.7071+0.7071j  0.0000+0.j      0.0000+0.j    ]
 [ 0.0000+0.j      0.7071-0.7071j  0.0000+0.j    ]
 [ 0.0000+0.j      0.0000+0.j      1.0000+0.j    ]]

S_inv =
[[ 0.70710678+0.j          0.00000000-0.70710678j  0.00000000+0.j        ]
 [ 0.70710678+0.j          0.00000000+0.70710678j  0.00000000+0.j        ]
 [ 0.00000000+0.j          0.00000000+0.j          1.00000000+0.j        ]]

S sigmaa S_inv =
[[ 0.7071 -3.76080387e-18j  0.7071 +1.04954358e-17j
   0.0000 +0.00000000e+00j]
 [-0.7071 +1.24975035e-17j  0.7071 +1.24975035e-17j
   0.0000 +0.00000000e+00j]
 [ 0.0000 +0.00000000e+00j  0.0000 +0.00000000e+00j
   1.0000 +0.00000000e+00j]]


#### Solving Linear Systems without Inverse

Let us try solving the following set of equations:

\begin{align*}
    x + y + z    &= 8 \\
    2x + 2y + 3z &= 15 \\
    x + 3y + 3z  &= 16
\end{align*}

This can be written using matrices and vectors as
\begin{equation*}
A\vec{x} = \vec{b}
\end{equation*}
where $A=
\begin{bmatrix}
        1 & 2 & 1 \\
        2 & 2 & 3 \\
        1 & 3 & 3 
\end{bmatrix}
\;\;\;\;\;\;
\vec{x} = 
\begin{bmatrix}
        x \\
        y \\
        z 
\end{bmatrix}
\;\;\;\;\;\;
\vec{b} = 
\begin{bmatrix}
        8 \\
        15 \\
        16
\end{bmatrix}$


Note that $A$ is a symmetric matrix. It has orthogonal eigenvectros.
<br> The matrix with eigenvectors of $A$ in columns is orthogonal.
<br> Its transpose and inverse are same.

In [18]:
A = np.array([[1, 2, 1], [2, 2, 3], [1, 3, 3]])
assert np.all(A == A.T)
b = np.array([8, 15, 16])

# One way to solve for this is to compute matrix inverse
# i.e x = A_inv b
x_0 = np.matmul(np.linalg.inv(A), b)
print("Solution using inverse: {}".format(x_0))

# Matrix inversion is a complex process that can be
# numerically unstable. If possible we use diagonalisation.
w, S = np.linalg.eig(A)

# We know that A = S sigma S_inv
# So S sigma S_inv x = b

# sigma S_inv x =  S_inv b  ==> sigma S_inv x = S_t b
S_inv_b = np.matmul(S.T, b)

# => S_inv x = sigma_inv(S_t b)
# Since inversion of the diagonal matrix is just division
# of all elements by 1, we can compute 
# sigma_inv as follows
sigma_inv = np.diag(1/ w)
sigma_inv_S_inv_b = np.matmul(sigma_inv, S_inv_b)

# => x = S (sigma_inv(S_t b))
x_1 = np.matmul(S, sigma_inv_S_inv_b) 
print("Solution using diagonalisation: {}".format(x_1))

assert np.allclose(x_0, x_1)

Solution using inverse: [ 1.  2.  3.]
Solution using diagonalisation: [ 1.  2.  3.]


#### Matrix powers using diagonalization

In [19]:
A = np.array([[1, 2, 1], [2, 2, 3], [1, 3, 3]])

def brute_force_matrix_power(matrix, y):
    """Returns matrix raised to the power of y"""
    # Assert that it is a square matrix
    assert len(matrix.shape) == 2\
           and matrix.shape[0] == matrix.shape[1] 
    assert type(y) == int 
    if y == 0:
        # Return identity matrix 
        return np.eye(matrix.shape[0])
    output_matrix = np.array(matrix, copy=True)
    for i in range(y-1):
        output_matrix = np.matmul(matrix, output_matrix)
    return output_matrix

def matrix_power_using_diag(matrix, y):
    """Returns matrix raised to the power of y"""
    # Assert that it is a square matrix
    assert len(matrix.shape) == 2\
           and matrix.shape[0] == matrix.shape[1] 
    assert type(y) == int 
    if y == 0:
        # Return identity matrix 
        return np.eye(matrix.shape[0])
    w, v = np.linalg.eig(matrix)
    w_y = w ** y # Compute w^y
    return np.matmul(v, np.matmul(np.diag(w_y),
                                  np.linalg.inv(v)))
    
# One way to compute A**2 would be to multiply A by A
# This is an O(nm^3) operation
A_2_brute_force = brute_force_matrix_power(A, 2)


# We can instead diagonalise the matrix and compute A ** 2
# This reduces the complexity to O(m)
A_2_diag = matrix_power_using_diag(A, 2)

print("A:\n{}".format(A))
print("A**2 computed via brute force:\n{}".\
      format(A_2_brute_force))
print("A**2 computed using diagonalisation:\n{}".\
      format(A_2_diag))

assert np.allclose(A_2_brute_force, A_2_diag)

A:
[[1 2 1]
 [2 2 3]
 [1 3 3]]
A**2 computed via brute force:
[[ 6  9 10]
 [ 9 17 17]
 [10 17 19]]
A**2 computed using diagonalisation:
[[  6.   9.  10.]
 [  9.  17.  17.]
 [ 10.  17.  19.]]
